<a href="https://colab.research.google.com/github/alexaK88/Q_jpeg_pennylane/blob/main/full_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane
!pip install pennylane pennylane-lightning[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.6/935.6 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.3/913.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.2/581.2 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.

### Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pennylane as qml
from pennylane.templates import QFT
from sklearn.svm import SVC
from sklearn.datasets import fetch_openml, load_digits
from sklearn.preprocessing import MinMaxScaler, normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from pennylane import numpy as pnp
from skimage.transform import resize
from keras.datasets import mnist

### Step 1:  Dataset Preparation

* Load MIST dataset - 0/1 pixel intensities
* Reduce image to 8x8
    * Resizing using skimage
    * Crop and pad (zero-padding)
* Normalize all images:
  8x8 image -> flatten -> vector of length 64 -> normalize
* Train/Test split

In [ ]:
# loading mnist from openML
mnist = fetch_openml('mnist_784', version=1, cache=True)
X = mnist['data'].astype(np.uint8) # better to convert for binerization
y = mnist['target'].astype(np.uint8)
y = y.to_numpy()

Here, I've been experimenting with different classes, and I stopped on 4 vs 9, cause they have more subtle difference in pixels, they are similar looking.

In [ ]:
# focus on binary classification
mask = (y == 4) | (y == 9)
X, y = X[mask], y[mask]
X.shape

(13782, 784)

In [ ]:
n_samples = 100 # restricting to 6000 samples for now

X = X.values if hasattr(X, "values") else X # safer conversion
perm = np.random.permutation(len(X))
X, y = X[perm], y[perm]

X = X[:n_samples]
y = y[:n_samples]

Normalise pixel intensities.

In [ ]:
X = X / 255.0
X = X.reshape(-1, 28, 28)

print(X.shape)
print("Pixel range:", X.min(), X.max())

(100, 28, 28)
Pixel range: 0.0 1.0


Reducing images to 8x8 + flattening to (, 64)

In [ ]:
# convert each 28x28 binarised image to 8x8, then flatten to length 64
def to_8x8_vector(img_row):
    img_8x8 = resize(
        img_row,
        (8, 8),
        anti_aliasing=False,
        preserve_range=True,
        order=1 # controlling interpolation
    )
    img_8x8 = img_8x8.flatten()
    s = np.sum(img_8x8)

    if s > 0:
        img_8x8 = np.sqrt(img_8x8 / s)
    else:
        img_8x8 = np.zeros_like(img_8x8)
        img_8x8[0] = 1.0
      # should be shape (64,)
    return img_8x8

# apply to all images
X_8x8 = np.array([to_8x8_vector(x) for x in X], dtype=float)
X_8x8.shape

(100, 64)

In [ ]:
# def safe_l2_normalize_rows(X):
#     norms = np.linalg.norm(X, axis=1, keepdims=True)
#     norms[norms == 0] = 1.0
#     return X / norms

# X_8x8_norm = safe_l2_normalize_rows(X_8x8)
print("Any NaNs?", np.isnan(X_8x8).any())
print("Norm check:", np.min(np.linalg.norm(X_8x8, axis=1)), np.max(np.linalg.norm(X_8x8, axis=1)))

Any NaNs? False
Norm check: 0.9999999999999999 1.0


I'm gonna do the splitting here, and carry both representations consistently

In [ ]:
idx = np.arange(n_samples)

idx_train, idx_test, y_train, y_test = train_test_split(
    idx, y, test_size=0.2, random_state=42, stratify=y
)

# QEK inputs (8x8 -> 64 -> normed)
X_train_qek = X_8x8[idx_train]
X_test_qek  = X_8x8[idx_test]

# QJPEG inputs (28x28 binary images)
X_train_img = X[idx_train]
X_test_img  = X[idx_test]

print("QEK train/test:", X_train_qek.shape, X_test_qek.shape)
print("IMG train/test:", X_train_img.shape, X_test_img.shape)
print("Labels train/test:", y_train.shape, y_test.shape)

QEK train/test: (80, 64) (20, 64)
IMG train/test: (80, 28, 28) (20, 28, 28)
Labels train/test: (80,) (20,)


Data preparation is done.

### Step 2: Quantum Embedding & Kernel Training

In [ ]:
n_qubits = 6
layers = 2
wires = range(n_qubits)

dev = qml.device("default.qubit", wires=wires, shots=None)

Defining QEK circuit

In [ ]:
@qml.qnode(dev, interface="autograd")
def kernel_qnode(x1, x2, theta):
    # Prepare |ψ(x1, θ)>
    qml.AmplitudeEmbedding(x1, wires=wires, normalize=False)

    for l in range(theta.shape[0]):
        for i in range(n_qubits):
            qml.RX(theta[l, i, 0], wires=i)
            qml.RY(theta[l, i, 1], wires=i)
            qml.RZ(theta[l, i, 2], wires=i)
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])

    # Apply adjoint of |ψ(x2, θ)>
    qml.adjoint(
        lambda: (
            qml.AmplitudeEmbedding(x2, wires=wires, normalize=False),
            *[
                (
                    qml.RX(theta[l, i, 0], wires=i),
                    qml.RY(theta[l, i, 1], wires=i),
                    qml.RZ(theta[l, i, 2], wires=i),
                )
                for l in range(theta.shape[0])
                for i in range(n_qubits)
            ],
        )
    )()

    # Probability of |00...0>
    return qml.expval(qml.Projector([0]*n_qubits, wires=wires))



option 1: train for 12 layers (max)
option 2: train for every number of layers

In [ ]:
n_qubits = 6
n_layers = 12          # number of trainable layers
layers = n_layers     # just to match variable names
batch_size = 12       # bigger batch for stability
n_steps = 50          # training steps
stepsize = 0.03       # smaller learning rate
eps_trace = 1e-8      # small epsilon to prevent division by zero

In [ ]:
def kernel_matrix(X1, X2, theta):
    return qml.math.stack([
        qml.math.stack([
            kernel_qnode(x1, x2, theta)
            for x2 in X2
        ])
        for x1 in X1
    ])


# --- Kernel alignment loss (batch) ---
def kernel_alignment_loss_batch(theta, X, y, batch_size=6):
    idx = np.random.choice(len(X), batch_size, replace=False)
    Xb = X[idx]
    yb = y[idx]

    # Kernel
    K = kernel_matrix(Xb, Xb, theta)

    # Labels: ±1
    y_pm = 2 * (yb == 9) - 1
    yy = qml.math.outer(y_pm, y_pm)

    # Center kernel
    n = batch_size
    H = qml.math.eye(n) - qml.math.ones((n, n)) / n
    Kc = H @ K @ H

    # Flatten
    Kf = qml.math.reshape(Kc, (-1,))
    yyf = qml.math.reshape(yy, (-1,))

    # Alignment = ⟨Kc, yy⟩ / (||Kc|| ||yy||)
    numerator = qml.math.dot(Kf, yyf)
    denominator = qml.math.sqrt(
        qml.math.dot(Kf, Kf) * qml.math.dot(yyf, yyf)
    )

    return -numerator / denominator


opt = qml.AdamOptimizer(stepsize=stepsize)

theta = 0.01 * pnp.random.randn(n_layers, n_qubits, 3, requires_grad=True)

def loss_fn(theta):
    return kernel_alignment_loss_batch(theta, X_train_qek, y_train, batch_size)

for step in range(n_steps):
    theta, loss = opt.step_and_cost(loss_fn, theta)
    print(f"Step {step}: loss = {loss:.4f}")

Step 0: loss = 0.1986
Step 1: loss = -0.4314
Step 2: loss = -0.0951
Step 3: loss = -0.0373
Step 4: loss = -0.4019
Step 5: loss = -0.4277
Step 6: loss = 0.0666
Step 7: loss = -0.3945
Step 8: loss = -0.2587
Step 9: loss = -0.3509
Step 10: loss = -0.4265
Step 11: loss = -0.5087
Step 12: loss = -0.2736
Step 13: loss = -0.4796
Step 14: loss = -0.3496
Step 15: loss = -0.3384
Step 16: loss = -0.3967
Step 17: loss = -0.5174
Step 18: loss = -0.2765
Step 19: loss = -0.2947
Step 20: loss = -0.6961
Step 21: loss = -0.2812
Step 22: loss = -0.5482
Step 23: loss = -0.4198
Step 24: loss = -0.4469
Step 25: loss = -0.7151
Step 26: loss = -0.3548
Step 27: loss = -0.2621
Step 28: loss = -0.4884
Step 29: loss = -0.6333
Step 30: loss = -0.3736
Step 31: loss = -0.0885
Step 32: loss = -0.5347
Step 33: loss = -0.2266
Step 34: loss = -0.5045
Step 35: loss = -0.4552
Step 36: loss = -0.3319
Step 37: loss = -0.7508
Step 38: loss = -0.7840
Step 39: loss = -0.7106
Step 40: loss = -0.6601
Step 41: loss = -0.6127
Step

In [26]:
theta_trained = theta
np.save("theta_trained.npy", theta_trained)

In [23]:
loss_trained = loss_fn(theta)
loss_zero = loss_fn(np.zeros_like(theta))

print(loss_trained, loss_zero)

-0.5908546858257661 0.327879369344936


In [ ]:
theta_zero = np.zeros_like(theta)
print(kernel_alignment_loss_batch(theta, X_train_qek, y_train, batch_size=16))
print(kernel_alignment_loss_batch(theta_zero, X_train_qek, y_train, batch_size=16))

-0.6426051027477397
0.11839052982375203


### Step 3: QJPEG Compression

In [ ]:
def vectorization(img, Cr, Cc, renorm=False):
    "Vectorize the image into amplitude-encoding patches suitable for quantum circuits"
    # splitting the original image (Mr, Mc) into S equal-size patches of shape (Cr, Cc)
    Mr, Mc = img.shape
    assert Mr % Cr == 0 and Mc % Cc == 0
    patches = (img.reshape(Mc//Cr, Cr, -1, Cc).swapaxes(1, 2).reshape(-1, Cr, Cc))
    # 64 patches, (64, 64, 64) shape; S=64

    # vectorize each patch and collect all in a (N, Cr*Cc) array
    vect_patches = np.reshape(patches,  (patches.shape[0], Cr*Cc)) # (64, 4096)

    # normalize each (Cr*Cc) vector to the intensity of the corresponding (Cr, Cc) patch
    states = np.zeros((patches.shape[0], Cr*Cc)) # (64, 4096)
    norm = np.zeros(patches.shape[0])

    for idx in range(patches.shape[0]): # for each patch
        # compute the sum of pixels intensities
        norm[idx] = vect_patches[idx].sum()
        if norm[idx] == 0:
            # empty patch -> encode |0...0>
            states[idx, 0] = 1.0
            norm[idx] = 1.0
            continue

        # normalize the patch vector so that its entries sum is 1
        tmp = vect_patches[idx] / norm[idx]
        # take the element-wise square root of the normalized vector
        states[idx] = np.sqrt(tmp)
    if renorm == False:
        norm = np.ones(patches.shape[0])
    print(states[:10])

    return states, norm # amplitudes, pixel intensities' sums

In [ ]:
def qft_swaps(wires):
    n = len(wires)
    # apply QFT to all qubits
    qml.QFT(wires=wires)
    # add swaps to reverse qubit order!
    for i in range(n // 2):
        qml.SWAP(wires=[wires[i], wires[n - i - 1]])


def iqft_swaps(wires):
    n = len(wires)
    # swaps again - BEFORE iqft
    for i in reversed(range(n // 2)):
        qml.SWAP(wires=[wires[i], wires[n-i-1]])
    qml.adjoint(QFT)(wires=wires)

In [ ]:
def circuit_builder(states, n0, n2, shots):
    ntilde = (n0 - n2) // 2
    n1 = n0 - ntilde

    qnodes = []

    # define device with n0 qubits
    dev = qml.device("lightning.qubit", wires=n0, shots=shots)

    for idx in range(states.shape[0]):
        # qnode to capture current input state
        @qml.qnode(dev)
        def circuit():
            # print("State norm:", np.linalg.norm(states[idx]))
            # initializing the state (using AmplitudeEmbedding here, but I'm wondering if something else could work faster)
            qml.AmplitudeEmbedding(states[idx], wires=range(n0), normalize=True)

            # Hadamard on all n0 qubits
            for w in range(n0):
                qml.Hadamard(wires=w)

            # apply QFT on all qubits
            qft_swaps(wires=range(n0))

            # apply IQFT on first n1 qubits
            iqft_swaps(wires=range(n1))

            # setting boundaries - Rule 2
            discard_start = n0 // 2 - ntilde
            discard_end = n0 // 2 - 1
            discarded_qubits = set(range(discard_start, discard_end + 1))

            # keep exactly n2 qubits for output
            measured_qubits = list(range(n2))


            # Hadamard on remaining qubits
            for q in measured_qubits:
                qml.Hadamard(wires=q)

            # print(f'Measured qubits: {measured_qubits}')

            return qml.probs(wires=measured_qubits)
        qnodes.append(circuit)

    return qnodes



In [ ]:
def reconstruction(qnodes, n2, norm):
    out_freq = np.zeros((len(qnodes), 2**n2))
    for idx, qnode in enumerate(qnodes):
        probs = qnode()
        out_freq[idx] = qnode() * norm[idx]

    return out_freq

In [ ]:
def devectorization(out_freq):
    S = out_freq.shape[0]
    nrow = int(np.sqrt(out_freq.shape[1])) # rows per patch
    ncol = nrow

    decoded_patches = np.reshape(out_freq,\
                      (out_freq.shape[0], nrow, ncol)) # (S, nrow, ncol)

    im_h, im_w = nrow*int(np.sqrt(S)), ncol*int(np.sqrt(S)) # final shape

    # initialization
    decoded_img = np.zeros((im_w, im_h))

    idx = 0
    for row in np.arange(im_h - nrow + 1, step=nrow):
        for col in np.arange(im_w - ncol + 1, step=ncol):
            decoded_img[row:row+nrow, col:col+ncol] = decoded_patches[idx]
            idx += 1

    return decoded_img

In [ ]:
def qjpeg_feature_map_quantum(img_28x28):
    """
    True QJPEG-inspired feature map:
    - probabilities sum to 1
    - amplitudes = sqrt(probabilities)
    - output dimension = 64 (6 qubits)
    """

    img = img_28x28.astype(float)
    img = img / img.sum()              # probabilities
    amps = np.sqrt(img.flatten())      # amplitudes

    # reduce to 64 amplitudes (simple truncation for now)
    amps = amps[:64]

    # safety
    if np.linalg.norm(amps) == 0:
        amps[0] = 1.0
    else:
        amps /= np.linalg.norm(amps)

    return amps


### Step 4: Inference without retraining

In [31]:
C = 1.0
layers_list = [2, 4, 6, 8, 10, 12]

dev = qml.device("default.qubit", wires=n_qubits, shots=None)

@qml.qnode(dev)
def qnode_state(x, theta):
    kernel_qnode(x, theta)
    return qml.state()

In [32]:
def compute_kernel(states_a, states_b=None):
    if states_b is None:
        states_b = states_a
    K = np.zeros((len(states_a), len(states_b)))
    for i, a in enumerate(states_a):
        for j, b in enumerate(states_b):
            K[i, j] = np.abs(np.vdot(a, b))**2
    return K

def evaluate_kernel_inference(
    qnode,
    theta,
    X_train,
    X_test,
    y_train,
    y_test,
    C=1.0,
):
    # 1. Compute quantum states
    states_train = np.array([qnode(x, theta) for x in X_train])
    states_test  = np.array([qnode(x, theta) for x in X_test])

    # 2. Kernel matrices
    K_train = compute_kernel(states_train)
    K_test  = compute_kernel(states_test, states_train)

    # 3. Normalize (important for SVM stability)
    max_val = np.max(K_train)
    if max_val > 0:
        K_train /= max_val
        K_test  /= max_val

    # 4. Classical SVM
    clf = SVC(kernel="precomputed", C=C)
    clf.fit(K_train, y_train)

    return accuracy_score(y_test, clf.predict(K_test))


In [35]:
theta_star = theta_trained  # frozen from Step 2

for layers in layers_list:
    print("\n" + "="*40)
    print(f"Evaluating {layers} layers")
    print("="*40)

    theta_L = theta_star[:layers]

    acc_qek = evaluate_kernel_inference(
        qnode_state,
        theta_L,
        X_train_qek,
        X_test_qek,
        y_train,
        y_test,
        C=C,
    )

    print(f"QEK accuracy: {acc_qek:.4f}")



Evaluating 2 layers
QEK accuracy: 0.9500

Evaluating 4 layers
QEK accuracy: 0.9500

Evaluating 6 layers
QEK accuracy: 0.9500

Evaluating 8 layers
QEK accuracy: 0.9500

Evaluating 10 layers
QEK accuracy: 0.9500

Evaluating 12 layers
QEK accuracy: 0.9500


In [37]:
print(np.linalg.norm(theta_trained[0] - theta_trained[1]))
print(np.linalg.norm(theta_trained[1] - theta_trained[2]))
print(np.linalg.norm(theta_trained[5] - theta_trained[6]))


1.618622001962192
1.7211565376730797


IndexError: index 5 is out of bounds for axis 0 with size 4

In [34]:
print(acc_qek)

0.95
